Process orders data 
- 1.Ingest the data into the data lakehouse - bronze.orders
- 2.perform data quality checks and transform the data as required - silver.orders_clean
- 3.Explode the items array from the order object -- silver.orders

In [0]:
create or refresh streaming table circuitbox.bronze.orders
comment "this table is raw data for order"
tblproperties ("Quality" = "bronze ")
as 
select *,
      _metadata.file_path as file_path,
      current_timestamp as ingested_timestamp
      from cloud_files(
        '/Volumes/circuitbox/landing/operational_data/orders/','json',
        map("cloudFiles.inferColumnTypes","true")
      )

2.Perform the data quality checks and transform the data as required - silver.orders_clean

In [0]:
CREATE OR REFRESH STREAMING TABLE circuitbox.silver.orders (
  CONSTRAINT valid_customer_id EXPECT (customer_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_order_id EXPECT (order_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_order_status EXPECT (order_status IN ('Pending', 'Shipped', 'Cancelled', 'Completed')),
  CONSTRAINT valid_payment_method EXPECT (payment_method IN ('Credit Card', 'Bank Transfer', 'Paypal'))
)
COMMENT 'This table is cleaned data for order'
TBLPROPERTIES ("quality" = "silver")
AS
SELECT
  order_id,
  customer_id,
  order_timestamp,
  payment_method,
  order_status,
  item.item_id,
  item.name AS item_name,
  item.price AS item_price,
  item.quantity AS item_quantity,
  item.category AS item_category
FROM (
  SELECT
    order_id,
    customer_id,
    CAST(order_timestamp AS TIMESTAMP) AS order_timestamp,
    payment_method,
    EXPLODE(items) AS item,
    order_status
  FROM STREAM(circuitbox.bronze.orders)
);